In [1]:
%matplotlib inline
from fifteen_sixteen_analysis import *

In [2]:
!ls ..

api_data_gov.key  bash_lib  data  LICENSE  nbs	notebooks  py_lib  README.md


In [3]:
# 2015-2016
CONTRIBUTIONS_COMMITTEE_INDIVS = "../data/itpas2.txt"
CANDIDATES = '../data/weball16.txt'
RACE = "G2016"
NUMBER_OF_COMPONENTS = 10
NEIGHBORS=10

import os.path
assert os.path.isfile(CONTRIBUTIONS_COMMITTEE_INDIVS)

In [4]:
sixteen_contri_df = read_contribution_tables(CONTRIBUTIONS_COMMITTEE_INDIVS)
candidates = read_candidates_table(CANDIDATES)


/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (10,11,12,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


In [5]:
rich_names = get_id_name_summed(
    contri_df=sixteen_contri_df,
    candid_df=candidates
)
filtered_df = filter_group_pivot(election_df=sixteen_contri_df, race=RACE)

nmf = sklearn.decomposition.NMF(n_components=NUMBER_OF_COMPONENTS)
tst = nmf.fit_transform(filtered_df.fillna(0))
kd = sp.kdtree.KDTree(tst)

In [6]:
def get_index_candidate(candidate_index, search_table):
    return search_table.iloc[candidate_index].name
    
def get_candidate_index(candidate_id, search_table):
    nimishs_df = search_table.reset_index()
    return nimishs_df[nimishs_df["candidate_id"] == candidate_id].index.values[0]

def query_candidate_id(candidate_id, search_table):
    can_index = get_candidate_index(candidate_id=candidate_id, search_table=search_table)
    return kd.query(tst[can_index], k=NEIGHBORS)

def get_contri_by_candidate_id(candidate_id):
    """Analysis function"""
    # Contributions
    contributions = sixteen_contri_df[['candidate_id', 'transaction_amount', 'contributor_name']]
    contributions = contributions.groupby(['candidate_id','contributor_name'])['transaction_amount'].sum().reset_index()
    # Candidates
    candidate_names = candidates[['CAND_ID', 'CAND_NAME']].drop_duplicates()

    # Merge 
    rich_fucks = pd.merge(contributions, candidate_names, left_on='candidate_id', right_on='CAND_ID')
    rich_fucks = rich_fucks.sort_values(by=['transaction_amount','contributor_name'], ascending=False).reset_index(drop=True)
    return rich_fucks[rich_fucks["candidate_id"] == candidate_id]

In [7]:
interesting_candidates = [
    "H6CA39020", # Flake
    "P60008059", # Bush
]

print(candidate_id, rich_names[rich_names["candidate_id"]==candidate_id].CAND_NAME)
distances, neighbor_indices = query_candidate_id(candidate_id=candidate_id, search_table=filtered_df)
print(distances)

for neighbor in neighbor_indices:
    neighbor_candidate_id = get_index_candidate(candidate_index=neighbor, search_table=filtered_df)
    print(neighbor_candidate_id, rich_names[rich_names["candidate_id"]==neighbor_candidate_id].CAND_NAME)

NameError: name 'candidate_id' is not defined

In [ ]:

flake_id="H6CA39020"
flake_can_index = get_candidate_index(candidate_id=flake_id, search_table=filtered_df)
bush_can_index = get_candidate_index(candidate_id=bush_id, search_table=filtered_df)
print(tst[flake_can_index])
bush_id = "P60008059"
print(tst[bush_can_index])
get_contri_by_candidate_id(bush_id)

In [ ]:
# find your candidate
# pull only candidate id and index
cand_id_index = filtered_df.index
nmf = sklearn.decomposition.NMF(n_components=10)
tst = nmf.fit_transform(filtered_df.fillna(0))
kd = sp.kdtree.KDTree(tst)
kd.query(tst[100], k=10)

# couple dweebs
candidates[candidates['CAND_ID'].isin([cand_id_index[100], cand_id_index[531]])]